<h1>Clustering Based On Max Projected Images</h1>
The idea is, instead of training the KMeans model with single frames from IC files, we can use max projection and reduce the X,Y,Z,T stacks to X,Y,Z stacks. This means around a 100-fold decrease in the memory requirement -because we won't need to hold thousands of frames per a TIFF stack in the memory, but only 32 images (as the Z dimensions' size is 32)-. I want to try this here.


The main suggestion would be using more than 32 clusters to over come the 3D depth differences between frames.In order to keep the spatial ordering, I encode every frame with its position on the Z-depth and because of that, if someone uses 32 clusters -or less- the clustering would probably be only based on that difference; to overcome this disadvantage I highly suggest using more -maybe many more-clusters to be able to separate depth encoding and real clustering. I may be wrong, this is only a hunch.

In [2]:
import glob
import nibabel as nib
import numpy as np
import os
import cv2

from sklearn.cluster import KMeans
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from ipywidgets import *
import ipywidgets as widgets


Setting the parameters for the whole run

In [18]:
#The r at the beginning of the string is important for path stuff
#The ** is for recursive search for folders
#The * is for any file with the ending 'IC.nii' , but you can write something else
IC_path = "\**\*IC.nii"

#Number of clusters for kmeans
n_clusters = 40

#Number of sample ICs to be used, 10 ICs take around 15-20 GB of RAM, so calculate accordingly
n_samples = 5

#IC resize resolution. All the 2D frames that make up the 4D nifti files will be resized to this resolution.
resize_resolution = (135,80)

#The file name and path the model that is going to be saved.If you want to save it in another folder,
#Change the path accordingly
km_model_file_name = "Models/km_model_"+str(n_clusters)+'_using'+str(n_samples)+"samples_and_max_projection.pkl"

#Do you want to save the model?
save_model = True

#If you want to plot cluster centers
plot_centers = False

#If you want to train a classifier?? ONLY WORKS WITH PAUL'S DATA FOR NOW
train_classifier = False


#If you want to plot cluster centers in depth order
plot_centers_in_depth_order = True



<h4>Finding the IC files</h4>

In [11]:
ic_paths = glob.glob(IC_path,recursive=True)
len(ic_paths)

286

Loading a sample of the IC files.

In [12]:
np.random.seed(0)
random_ic_paths = np.random.choice(ic_paths,n_samples)

In [13]:
random_ics = []
for ic_file in random_ic_paths:
    ic = nib.load(ic_file)
    random_ics.append(ic)

In [14]:

#Slice the ics frames and put them in an array
random_ics_extracted_and_flattened_frames = {}
for ic in random_ics:
    random_ics_extracted_and_flattened_frames[ic.get_filename()] = []
    ic_data = ic.get_fdata().T
    max_projected_ic_data = ic_data.max(axis=0)
    for i,component in enumerate(max_projected_ic_data):
        #Resize the component to 80x135, the value is arbitrary, all the IC files I have seen are 135 pixels wide,the row count changes
        #It may be changed
        resized_component = cv2.resize(component, resize_resolution, interpolation=cv2.INTER_AREA)
        #adding a 1000 times slice index so that we can hopefully keep track of the slice positions
        resized_component = resized_component + 1000*i
        #Reshape the 2D array to 1D array, because KMeans can't do 2d
        random_ics_extracted_and_flattened_frames[ic.get_filename()].append(resized_component.reshape(resized_component.shape[0]*resized_component.shape[1]))


In [15]:
flattened_image_size = resize_resolution[0]*resize_resolution[1]
array_for_clustering = np.empty((0,flattened_image_size))
for key in random_ics_extracted_and_flattened_frames.keys():
    #concatenate array_to_added to array for clustering
    array_for_clustering = np.concatenate((array_for_clustering,np.array(random_ics_extracted_and_flattened_frames[key])),axis=0)
    random_ics_extracted_and_flattened_frames[key] =''
array_for_clustering.shape

(157, 10800)

In [19]:
km = KMeans(n_clusters=n_clusters,random_state=42,n_init="auto")
km = km.fit(array_for_clustering)


if save_model:
    with open(km_model_file_name,"wb") as f:
        pickle.dump(km,f)

c:\Users\mreki\miniconda3\envs\ic_clustering\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Displaying the cluster centers. Warning:THESE ARE NOT ORDERED BY DEPTH.

In [20]:
if plot_centers:
# The '//' mean it is an integer division, so the result is an integer
    fig, ax = plt.subplots(n_clusters//4,4, figsize=(4*4,n_clusters))
    centers = km.cluster_centers_.reshape(n_clusters,-1,resize_resolution[0])



    for i, (axi, center) in enumerate(zip(ax.flat, centers)):
        axi.set(xticks=[], yticks=[])
        axi.imshow(center, interpolation='nearest', cmap='gray')
        axi.set_title("Cluster "+str(i) + "Slice: "+str(center.mean()//1000))

This function is only necessary for training a classifier on Paul's data. It finds experimental conditions based on file names.

In [57]:
def create_ic_paths_and_conditions_df(ic_paths):
    ic_paths_and_conditions = {"ic_paths":ic_paths,"starved_fed":[],"odor_taste":[]}
    for ic_path in ic_paths:
        if "Fed" in ic_path:
            ic_paths_and_conditions["starved_fed"].append("Fed")
        else:
            ic_paths_and_conditions["starved_fed"].append("Starved")

        if "Odor" in ic_path:
            ic_paths_and_conditions["odor_taste"].append("Odor")
            continue
        elif "Taste" in ic_path:
            ic_paths_and_conditions["odor_taste"].append("Taste")
            continue
        else:
            ic_paths_and_conditions["odor_taste"].append("Multi")
            continue
    return pd.DataFrame(ic_paths_and_conditions)


This function is also only necessary for Paul's data -for now- as it is used to create a training dataframe for classifier training. It could be extended to be used with other experiments.

In [58]:
def create_training_df(random_ics,km):

    max_value_for_component_for_each_IC = {}
    for ic in random_ics:
        max_value_for_component_for_each_IC[ic.get_filename()] = {}
    print(max_value_for_component_for_each_IC.keys())

    for ic in random_ics:
        max_value_for_component_for_current_IC = {}
        for key in range(n_clusters):
            max_value_for_component_for_current_IC[key] = 0
        ic_data = ic.get_fdata().T
        max_projected_ic_data = ic_data.max(axis=0)

        
        for i,frame in enumerate(max_projected_ic_data):
            #We match the shape of the data that the kmeans was trained with
            #flattened_frame = frame.reshape(frame.shape[0]*frame.shape[1])[0:km.cluster_centers_.shape[1]]
            #The 1000*i is for encoding the depth value of the frame
            flattened_frame = cv2.resize(frame, (80,135),interpolation=cv2.INTER_AREA).reshape(135*80)+1000*i
            predicted_cluster = km.predict([flattened_frame])
            max_value_of_this_frame = np.max(flattened_frame)
            if max_value_of_this_frame > max_value_for_component_for_current_IC[predicted_cluster[0]]:
                max_value_for_component_for_current_IC[predicted_cluster[0]] = max_value_of_this_frame
        max_value_for_component_for_each_IC[ic.get_filename()] = max_value_for_component_for_current_IC

    max_value_for_component_for_each_IC_df = pd.DataFrame(max_value_for_component_for_each_IC)
    max_value_df = max_value_for_component_for_each_IC_df.T.reset_index().rename(columns={"index":"IC_file_path"})
    random_ic_paths_and_conditions_df = create_ic_paths_and_conditions_df(max_value_df['IC_file_path'])
    print(len(random_ic_paths_and_conditions_df))
    training_df = pd.merge(random_ic_paths_and_conditions_df,max_value_df,left_on="ic_paths",right_on="IC_file_path").drop(columns="IC_file_path")



    return training_df

In [76]:
if train_classifier:
    batch_size = 13
    batch_ics = []
    for ic_file in ic_paths[0:batch_size]:
        ic = nib.load(ic_file)
        batch_ics.append(ic)
    training_df = create_training_df(batch_ics,km)
    for i in range(batch_size,len(ic_paths),batch_size):
            batch_ics = []
            for ic_file in ic_paths[i:i+batch_size]:
                ic = nib.load(ic_file)
                batch_ics.append(ic)
            training_df = pd.concat([training_df,create_training_df(batch_ics,km)])
            print(len(training_df))
            training_df.to_csv('DataframesFromClustering/training_df_'+str(len(km.cluster_centers_))+'clusters_max_projected.csv')
    #Train a random forest with training 
    X=training_df.drop(columns=["ic_paths","starved_fed","odor_taste"])
    y_odor = training_df["odor_taste"]
    rf = RandomForestClassifier(n_estimators = 5000)

    #Do a train test split and see the accuracy
    X_train, X_test, y_train, y_test = train_test_split(X, y_odor, test_size=0.3, random_state=42)

    rf.fit(X_train,y_train)
    y_pred = rf.predict(X_test)
    print("Accuracy for model with ",len(km.cluster_centers_)," clusters is ",accuracy_score(y_test,y_pred))


dict_keys(["//172.24.64.12/data/gklab/OgulcanCingiler/Paul's dff Files/Multi Sense Aug 22 (dFF data)/Excluded/DataPaul497ss1regdFF60skf256Smith20_4_129IC.nii", "//172.24.64.12/data/gklab/OgulcanCingiler/Paul's dff Files/Multi Sense Aug 22 (dFF data)/Excluded/DataPaul498ss1regdFF60skf212Smith20_4_100IC.nii", "//172.24.64.12/data/gklab/OgulcanCingiler/Paul's dff Files/Multi Sense Aug 22 (dFF data)/Excluded/DataPaul499ss1regdFF60skf272Smith20_4_132IC.nii", "//172.24.64.12/data/gklab/OgulcanCingiler/Paul's dff Files/Multi Sense Aug 22 (dFF data)/Fed/DataPaul482ss1regdFF60skf256Smith20_4_140IC.nii", "//172.24.64.12/data/gklab/OgulcanCingiler/Paul's dff Files/Multi Sense Aug 22 (dFF data)/Fed/DataPaul483ss1regdFF60skf202Smith20_4_106IC.nii", "//172.24.64.12/data/gklab/OgulcanCingiler/Paul's dff Files/Multi Sense Aug 22 (dFF data)/Fed/DataPaul484ss1regdFF60skf216Smith20_4_122IC.nii", "//172.24.64.12/data/gklab/OgulcanCingiler/Paul's dff Files/Multi Sense Aug 22 (dFF data)/Fed/DataPaul485ss1re

This function is necessary for displaying cluster centers in depth order, so that the bad clusters can be selected

In [21]:

def display_cluster_centers_in_depth_order(cluster_centers,n=0):
    #cluster_centers = km.cluster_centers_

    row_idxs = np.argsort(np.mean(cluster_centers * -1, axis=1))

    sorted_cluster_centers = cluster_centers[row_idxs]
    center = sorted_cluster_centers[n]
    
    plt.imshow(center.reshape(resize_resolution[1],resize_resolution[0]), interpolation='nearest', cmap='gray')
    plt.title("Cluster "+str(row_idxs[n]) + "Slice: "+str(center.mean()//1000))

In [22]:
#Make an interactive plot using ipywidgets so that we can see the cluster centers in depth order with a slider to select
#the cluster
if plot_centers_in_depth_order:
    interact(display_cluster_centers_in_depth_order,cluster_centers=fixed(km.cluster_centers_),n=(0,len(km.cluster_centers_)-1))

interactive(children=(IntSlider(value=0, description='n', max=39), Output()), _dom_classes=('widget-interact',…

<h4>Here we select the bad and good clusters </h4>
The idea is, after the selection of -either the good or the bad- clusters the code can go over all the images in all the IC files and discard the bad ones and save only the good ones


In [23]:
#I selected some clusters as the bad ones
bad_clusters = [1,5,10,190]


Deleting all the bad ICs

In [24]:
def clean_and_save_ics(ics,bad_clusters,km,clean_ic_folder):

    for ic in ics:
        ic_data = ic.get_fdata().T
        max_projected_ic_data = ic_data.max(axis=0)
        cleaned_ic = []

        #Here we create a filename based on the original file.You can change it as you like
        cleaned_ic_path = clean_ic_folder +'/'+'_'.join(ic.get_filename().replace('.nii','').split('/')[-3:])+'MAX_PROJECTED_CLEANED.nii'
        for i,frame in enumerate(max_projected_ic_data):
            #We match the shape of the data that the kmeans was trained with
            #flattened_frame = frame.reshape(frame.shape[0]*frame.shape[1])[0:km.cluster_centers_.shape[1]]
            #The 1000*i is for encoding the depth value of the frame
            flattened_frame = cv2.resize(frame, (80,135),interpolation=cv2.INTER_AREA).reshape(135*80)+1000*i
            predicted_cluster = km.predict([flattened_frame])
           
            if predicted_cluster not in bad_clusters:
                cleaned_ic.append(frame)
        cleaned_ic = np.array(cleaned_ic)
        img = nib.Nifti1Image(cleaned_ic.T, np.eye(4))
        nib.save(img,cleaned_ic_path )    

Below I only use random ICs to clean and save the IC files based on the bad ics list, but one can use all the ics by loading them like:

```python
all_ics = []
for ic_file in ics: 
    ic = nib.load(ic_file) 
    all_ics.append(ic)
    

In [25]:
clean_and_save_ics(random_ics,bad_clusters,km,'CleanedICs')